In [188]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from keras import backend as K
from scipy.spatial import distance

def ComputeMaxDistance(X1, X2):
    max_dist = 0
    for x1 in X1:
        for x2 in X2:
            dist = distance.euclidean(x1, x2)
            max_dist = max(dist, max_dist)
    return max_dist

def RBF_kernel(x, centers, sigma):
  # res = []
  # for c in centers:
    # res.append(math.exp((-np.linalg.norm(x - c)**2) / (2*sigma**2)))
  # res = [math.exp(-np.linalg.norm(x - c)**2) / (2 * sigma**2) for c in centers]
  res = [math.exp(-np.linalg.norm(x - c)**2 / (2 * sigma**2)) for c in centers]
  return res

def RBF_layer_output(x, centers, sigma):
  # res = []
  # for x_ in x: 
  #   res.append(RBF_kernel(x_, centers, sigma))
  # res = [RBF_kernel(x_, centers, sigma) for x_ in x]
  # res = [0 for x_ in x]
  res = [RBF_kernel(x_, centers, sigma) for x_ in x]

  return res

class RBF(keras.layers.Layer):

  def __init__(self, num_units):
    self.num_units = num_units
    super(RBF, self).__init__()

  def build(self, input_shape):
    # shape1 = tf.TensorShape((input_shape[1], self.num_units))
    # shape2 = tf.TensorShape((1, self.num_units))
    self.centers = self.add_weight(shape=(input_shape[1], self.num_units), initializer=tf.initializers.RandomNormal, trainable=False) # TODO use custom initializer
    self.sigma = self.add_weight(shape=(1, self.num_units), initializer=tf.initializers.RandomNormal, trainable=False) # TODO use custom initializer
    # self.betas = np.ones(self.num_units) / (2 * (sigma**2))
    # self.sigma    = self.add_weight(shape=1, initializer=tf.initializers.RandomNormal, trainable=False)
    super(RBF, self).build(input_shape)
  
  def call(self, inputs):
    D = inputs - tf.transpose(self.centers)
    print(D)
    return tf.exp(-tf.norm(D, axis=1)**2) / (2 * self.sigma[0]**2) # TODO change sigma[0]

  def get_config(self):
    base_config = super(RBF, self).get.config
    base_config['num_units'] = self.num_units
    return base_config

  def compute_params(self, inputs):
    kmeans = KMeans(n_clusters=self.num_units, random_state=0, copy_x=True).fit(inputs)
    self.centers = kmeans.cluster_centers_
    d_max = ComputeMaxDistance(self.centers, self.centers)
    self.sigma = d_max / (math.sqrt(2 * self.centers.shape[0]))



In [164]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import preprocessing
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
import utils
import rbf_layer

# ---------------------------------------------------------------------------- #
#                                   read data                                  #
# ---------------------------------------------------------------------------- #

# data_url = "http://lib.stat.cmu.edu/datasets/boston"
data = "boston.csv"
raw_df = pd.read_csv(data, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

# ---------------------------------------------------------------------------- #
#                                normalize data                                #
# ---------------------------------------------------------------------------- #

preprocessing.scale(X, copy=False)
preprocessing.scale(y, copy=False)

# ---------------------------------------------------------------------------- #
#                      split data to training and testing                      #
# ---------------------------------------------------------------------------- #

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)

n_train = X_train.shape[0]
hidden_size = int(0.1 * n_train)

In [189]:
rbf_layer_ = RBF(hidden_size)
rbf_layer_.compute_params(X_train)
# rbf_layer_.compute_params(X_train)

from keras import layers

model = keras.Sequential()
model.add(Input(shape=(13,)))
# model.add(layers.Dense(12, input_shape=(13,), activation='relu'))
model.add(rbf_layer_)
# model.add(layers.Dense(8, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))


Tensor("rbf_48/sub:0", shape=(37, 13), dtype=float32)


In [182]:
# model.compile(optimizer=tf.keras.optimizers.Adam(),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
#               metrics=['accuracy'])

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile()
# model.fit(X_train, y_train, epochs=5, batch_size=10)

# train model
# history = model.fit(X_train, y_train)


In [187]:
# print(history.params)
# model.summary()
model.predict(X_train[0])
# a = tf.constant([1, 2])
# print(type(a))

array([1.5356340e-01, 3.4652334e-02, 2.0106247e-02, 3.4650344e-02,
       1.9822607e-02, 6.2260521e-03, 4.9070187e-02, 1.1567194e-01,
       3.8614171e+00, 8.3840352e-01, 4.4612970e-02, 1.2057885e-02,
       2.3655333e-01, 4.2094436e-02, 8.3106123e-02, 1.0509519e-02,
       3.6319152e-01, 2.8838949e+01, 1.6627885e+00, 3.3018343e-02,
       1.2861995e-01, 8.5926950e-02, 4.7045428e-02, 3.6514189e-02,
       6.5708494e-01, 1.0345719e-01, 8.2592636e-02, 1.2584703e+02,
       4.4353214e-01, 1.6439890e-02, 3.3533174e-01, 4.1287810e-01,
       3.7816346e-02, 1.4827551e-02, 1.7922504e-01, 4.5908085e+01,
       1.2840350e-01], dtype=float32)

In [128]:

# t = tf.constant([3, 2])

x = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
centers = tf.constant([[1, 2, 3], [4, 5, 6]])

# for x_ in x:
#     for c in centers:
#         print(-np.linalg.norm(c - x_)**2 / 2)

for x_ in x:
    print(x_-centers)

# print([(-np.linalg.norm(c - x)**2 / 2) for c in centers])

# x = tf.expand_dims(tf.constant([[1, 2, 3],[4, 5, 6]]), -1)
# print(x.numpy())

# centers = tf.constant([[1, 2, 3], [4, 5, 6]])

# sigma = 1

# res = tf.constant([math.exp(-np.linalg.norm(x - c)**2 / (2 * sigma**2)) for c in centers])
# print(res)

# for t_ in t:
#     print(t_)

tf.Tensor(
[[ 0  0  0]
 [-3 -3 -3]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[3 3 3]
 [0 0 0]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[6 6 6]
 [3 3 3]], shape=(2, 3), dtype=int32)
